# Import library

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import sklearn
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

# Plot confusion matrix

In [4]:
def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    Parameter
    ---------
    cm : confusion matrix
    classes : various category
    normalize : True, False
    title : you can type anthing you in it, but the data type must be "string"
    default 'Confusion matrix'
    cmap : default 'plt.cm.Blues'
    reference : https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment = "center",
                     color = "white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# load data intra subject

In [5]:
def load_data(time, condition):
    
    """
    This function is used for load csv file
    from same folder
    
    Parameters
    ----------
    time : '2s', '1s', '0.5s'
    condition : 'stationary', 'holding'
    
    Returns
    -------
    feature_df : dataframe
    rf_features : array-like of shape (number_of_samples, number_of_features)
    without normalize
    svm_features : array-like of shape (number_of_samples, number_of_features)
    with normalize
    label : label of each category
    feature_column : name of each feature
    """
    
    gestures = ['pat', 'stroke', 'grab', 'poke', 'scratch', 'notouch']
    path = '/content/drive/My Drive/研究資料(共用)/svm_rf_feature_s1s25/' + condition + '/'
    data = np.zeros((1, 27))
    gesture_type = 1
    labels = []
    if condition == 'stationary':
        for gesture in gestures:
            gesture_df = pd.read_csv(os.path.join(path, gesture + '_s_' + time + '_' + 's1s25.csv'))
            gesture_df.drop(['Unnamed: 0'], inplace = True, axis = 1)
            features_column = gesture_df.columns.values
            gesture_df = SimpleImputer().fit_transform(gesture_df)
            data = np.concatenate((data, gesture_df))
            label= [gesture_type for i in range(len(gesture_df))]
            labels = np.concatenate((labels, label), axis = 0)
            gesture_type += 1
    else:
        for gesture in gestures:
            gesture_df = pd.read_csv(os.path.join(path, gesture + '_h_' + time + '_' + 's1s25.csv'))
            gesture_df.drop(['Unnamed: 0'], inplace = True, axis = 1)
            features_column = gesture_df.columns.values
            gesture_df = SimpleImputer().fit_transform(gesture_df)
            data = np.concatenate((data, gesture_df))
            label= [gesture_type for i in range(len(gesture_df))]
            labels = np.concatenate((labels, label), axis = 0)

    data = data[1:]
    labels = labels.astype('int')
    features_df = pd.DataFrame(data, columns = features_column)
    rf_features = np.array(data)
    svm_features = MinMaxScaler().fit_transform(features_df)
    
    return features_df, rf_features, svm_features, labels, features_column
        

# load data inter subject

In [6]:
def load_random_subject(time, condition_1, condition_2, folder):
    """
    This function is used for load csv file
    from various folder
    
    Parameters
    ----------
    time : '2s', '1s', '0.5s'
    condition_1 : 'stationary', 'holding'
    condition_2 : 'train', 'test'
    folder : 1, 2, 3, ...10
    
    Returns
    -------
    feature_df : dataframe
    rf_features : array-like of shape (number_of_samples, number_of_features)
    without normalize
    svm_features : array-like of shape (number_of_samples, number_of_features)
    with normalize
    label : label of each category
    feature_column : name of each feature
    """
    gestures = ['pat', 'stroke', 'grab', 'poke', 'scratch', 'notouch']
    path = '/content/drive/My Drive/研究資料(共用)/different subject train test/' + condition_1 + '_s1s25/' + 'cv_' + str(folder) + '/feature_cv_' + str(folder)
    data = np.zeros((1, 27))
    gesture_type = 1
    labels = []
    if condition_1 == 'stationary':
        for gesture in gestures:
            csv_file_name = condition_2 + '_' + gesture + '_s_' + time + '_s1s25_' + 'cv_' + str(folder) + '.csv'
            gesture_df = pd.read_csv(os.path.join(path, csv_file_name))
            gesture_df.drop(['Unnamed: 0'], inplace = True, axis = 1)
            features_column = gesture_df.columns.values
            gesture_df = SimpleImputer().fit_transform(gesture_df)
            data = np.concatenate((data, gesture_df))
            label= [gesture_type for i in range(len(gesture_df))]
            labels = np.concatenate((labels, label), axis = 0)
            gesture_type += 1
    else:
        for gesture in gestures:
            csv_file_name = condition_2 + '_' + gesture + '_h_' + time + '_s1s25_' + 'cv_' + str(folder) + '.csv'
            gesture_df = pd.read_csv(os.path.join(path, csv_file_name))
            gesture_df.drop(['Unnamed: 0'], inplace = True, axis = 1)
            features_column = gesture_df.columns.values
            gesture_df = SimpleImputer().fit_transform(gesture_df)
            data = np.concatenate((data, gesture_df))
            label= [gesture_type for i in range(len(gesture_df))]
            labels = np.concatenate((labels, label), axis = 0)
            gesture_type += 1
    
    data = data[1:]
    labels = labels.astype('int')
    features_df = pd.DataFrame(data, columns = features_column)
    rf_features = np.array(data)
    svm_features = MinMaxScaler().fit_transform(features_df)
    
    return features_df, rf_features, svm_features, labels, features_column 

# Random select data

In [7]:
def random_select(s_data, s_olabel, h_data, h_olabel):
    """
    This function can shuffle the data
    
    Parameter
    ---------
    s_data: data from stationary dataset
    s_olabel : label from stationary dataset
    h_data: data from holding dataset
    h_olabel : label from holding dataset
    
    Returns
    -------
    half of numbers of each posture dataset
    """
    len_s = len(s_olabel)
    len_h = len(h_olabel)
    s_randomize = np.arange(len_s)
    h_randomize = np.arange(len_h)
    np.random.shuffle(s_randomize)
    np.random.shuffle(h_randomize)
    half_s_data = s_data[s_randomize[:int(0.5 * len_s)]]
    half_h_data = h_data[h_randomize[:int(0.5 * len_h)]]
    half_s_olabel = s_olabel[s_randomize[:int(0.5 * len_s)]]
    half_h_olabel = h_olabel[h_randomize[:int(0.5 * len_h)]]
    
    
    return half_s_data, half_s_olabel, half_h_data, half_h_olabel

# Concatenate data

In [8]:
def concatenate(s_features, h_features, s_labels, h_labels):
    """
    This function can concatenate the data
    
    Parameter
    ---------
    s_features: data from stationary dataset
    s_labels : label from stationary dataset
    h_data: data from holding dataset
    h_labels : label from holding dataset
    
    Returns
    -------
    t_features : features which concateneate from each posture dataset
    t_labels : labels which concateneate from each posture dataset 
    """
    t_features = np.concatenate((s_features, h_features))
    t_labels = np.concatenate((s_labels, h_labels))
    
    return t_features, t_labels

# Intra subject (same posture)

In [9]:
# SVM
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
phase = ['stationary', 'holding']
#--------------------------------------------------------------------------------
for c in phase:
    conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
    for length in times:
        print('Now start 10 fold cross validation')
        print('phase = ' + c)
        print('data_length = ' + length)
        cvscores = []
        i = 1
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
        features_df, rf_features, svm_features, labels, features_column = load_data(length, c)
        k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)
        for train, validate in k_fold.split(svm_features, labels):
            print('Now, start the cv: ',i)
            x_train, y_train = svm_features[train], labels[train]
            x_validate, y_validate = svm_features[validate], labels[validate]
            svm = SVC(kernel = 'rbf', C = 8, gamma = 32, decision_function_shape = 'ovo')
            svm.fit(x_train, y_train)
            pred = svm.predict(x_validate)
            scores = accuracy_score(y_validate, pred)* 100
            print("The accuracy in this fold: %f"%scores)
            print("=============================")
            cvscores.append(scores)
            conf_mat = np.expand_dims(confusion_matrix(y_validate, pred), axis = 2)
            conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
            i += 1
#--------------------------------------------------------------------------------
# this part is to print out the average accuracy
#--------------------------------------------------------------------------------
        print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
        print("=============================================")
        print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------   
    conf_mat_sum = np.sum(conf_mat_all, axis = 2)
    plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                      normalize = True)
    # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + c + '_intra subject confusion_matrix.png')
    plt.show()

Now start 10 fold cross validation
phase = stationary
data_length = 2s


FileNotFoundError: ignored

In [ ]:
# Random Forest
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
phase = ['stationary', 'holding']
#--------------------------------------------------------------------------------
for c in phase:
    conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
    for length in times:
        print('Now start 10 fold cross validation')
        print('phase = ' + c)
        print('data_length = ' + length)
        cvscores = []
        i = 1
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
        features_df, rf_features, svm_features, labels, features_column = load_data(length, c)
        k_fold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)
        for train, validate in k_fold.split(svm_features, labels):
            print('Now, start the cv: ',i)
            x_train, y_train = rf_features[train], labels[train]
            x_validate, y_validate = rf_features[validate], labels[validate]
            rf = RandomForestClassifier(n_estimators = 70, max_depth = 10, min_samples_leaf = 4, n_jobs = -1)
            rf.fit(x_train, y_train)
            pred = rf.predict(x_validate)
            scores = accuracy_score(y_validate, pred)* 100
            print("The accuracy in this fold: %f"%scores)
            print("=============================")
            cvscores.append(scores)
            conf_mat = np.expand_dims(confusion_matrix(y_validate, pred), axis = 2)
            conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
            i += 1
#--------------------------------------------------------------------------------
# this part is to print out the average accuracy
#--------------------------------------------------------------------------------
        print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
        print("=============================================")
        print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
    conf_mat_sum = np.sum(conf_mat_all, axis = 2)
    plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                      normalize = True)
    # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + c + '_intra subject confusion_matrix.png')
    plt.show()

Now start 10 fold cross validation
phase = stationary
data_length = 2s
Now, start the cv:  1
The accuracy in this fold: 77.792732
Now, start the cv:  2
The accuracy in this fold: 78.196501
Now, start the cv:  3
The accuracy in this fold: 79.273217
Now, start the cv:  4
The accuracy in this fold: 78.706199
Now, start the cv:  5
The accuracy in this fold: 76.819407
Now, start the cv:  6
The accuracy in this fold: 77.897574
Now, start the cv:  7
The accuracy in this fold: 80.053908
Now, start the cv:  8
The accuracy in this fold: 80.727763
Now, start the cv:  9
The accuracy in this fold: 77.493261
Now, start the cv:  10
The accuracy in this fold: 79.784367
2s Total Accuracy (average):  78.67449293133032 1.189799650833501
Now start 10 fold cross validation
phase = stationary
data_length = 1s
Now, start the cv:  1
The accuracy in this fold: 78.047138
Now, start the cv:  2
The accuracy in this fold: 77.441077
Now, start the cv:  3
The accuracy in this fold: 77.777778
Now, start the cv:  4
Th

KeyboardInterrupt: ignored

# Intra Subjcet (different posture)

In [ ]:
# SVM
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
for length in times:
    print('data_length = ' + length)
    s_features_df, s_rf_features, s_svm_features, s_labels, s_features_column = load_data(length, 'stationary')
    h_features_df, h_rf_features, h_svm_features, h_labels, h_features_column = load_data(length, 'holding')
    cvscores = []
    print('Now start 10 fold cross validation')
    for i in range(1, 11):
        print('Now, start the cv: ',i)
        train_s_svm_features, test_s_svm_features, train_s_labels, test_s_labels = train_test_split(s_svm_features, s_labels, test_size = 0.1, shuffle = True)
        train_h_svm_features, test_h_svm_features, train_h_labels, test_h_labels = train_test_split(h_svm_features, h_labels, test_size = 0.1, shuffle = True)
        svm = SVC(kernel = 'rbf', C = 8, gamma = 32, decision_function_shape = 'ovo')
        svm.fit(train_s_svm_features, train_s_labels)
        pred = svm.predict(test_h_svm_features)
        scores = accuracy_score(test_h_labels, pred) * 100
        print('The accuracy in  this fold: %f'%scores)
        conf_mat = np.expand_dims(confusion_matrix(test_h_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
        cvscores.append(scores)
    print(length + " Total Accuracy with SVM (average): ", np.mean(cvscores), np.std(cvscores))
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues, normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + '_train stationary test holding confusion_matrix.png')
plt.show()

In [ ]:
# SVM
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
for length in times:
    print('data_length = ' + length)
    s_features_df, s_rf_features, s_svm_features, s_labels, s_features_column = load_data(length, 'stationary')
    h_features_df, h_rf_features, h_svm_features, h_labels, h_features_column = load_data(length, 'holding')
    print('Now start 10 fold cross validation')
    cvscore = []
    for i in range(1, 11):
        print('Now, start the cv: ',i)
        train_s_svm_features, test_s_svm_features, train_s_labels, test_s_labels = train_test_split(s_svm_features, s_labels, test_size = 0.1, shuffle = True)
        train_h_svm_features, test_h_svm_features, train_h_labels, test_h_labels = train_test_split(h_svm_features, h_labels, test_size = 0.1, shuffle = True)
        svm = SVC(kernel = 'rbf', C = 8, gamma = 32, decision_function_shape = 'ovo')
        svm.fit(train_h_svm_features, train_h_labels)
        pred = svm.predict(test_s_svm_features)
        scores = accuracy_score(test_s_labels, pred) * 100
        print('The accuracy in  this fold: %f'%scores)
        conf_mat = np.expand_dims(confusion_matrix(test_s_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
        cvscores.append(scores)
    print(length + " Total Accuracy with SVM (average): ", np.mean(cvscores), np.std(cvscores))
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues, normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + '_train holding test stationary confusion_matrix.png')
plt.show()

In [ ]:
# RF
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
for length in times:
    cvscores = []
    print('data_length = ' + length)
    s_features_df, s_rf_features, s_svm_features, s_labels, s_features_column = load_data(length, 'stationary')
    h_features_df, h_rf_features, h_svm_features, h_labels, h_features_column = load_data(length, 'holding')
    print('Now start 10 fold cross validation')
    for i in range(1, 11):
        print('Now, start the cv: ',i)
        train_s_rf_features, test_s_rf_features, train_s_labels, test_s_labels = train_test_split(s_rf_features, s_labels, test_size = 0.1, shuffle = True)
        train_h_rf_features, test_h_rf_features, train_h_labels, test_h_labels = train_test_split(h_rf_features, h_labels, test_size = 0.1, shuffle = True)
        rf = RandomForestClassifier(n_estimators = 70, max_depth = 10, min_samples_leaf = 4, n_jobs = -1)
        rf.fit(train_s_rf_features, train_s_labels)
        pred = rf.predict(test_h_rf_features)
        scores = accuracy_score(test_h_labels, pred) * 100
        print('The accuracy in  this fold: %f'%scores)
        conf_mat = np.expand_dims(confusion_matrix(test_h_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
        cvscores.append(scores)
    print(length + " Total Accuracy with RF (average): ", np.mean(cvscores), np.std(cvscores))
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues, normalize = True)
# plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + '_train stationary test holding confusion_matrix.png')
plt.show()

data_length = 2s
Now start 10 fold cross validation
Now, start the cv:  1
The accuracy in  this fold: 19.623656
Now, start the cv:  2
The accuracy in  this fold: 19.220430
Now, start the cv:  3
The accuracy in  this fold: 20.430108
Now, start the cv:  4
The accuracy in  this fold: 19.489247
Now, start the cv:  5
The accuracy in  this fold: 21.908602
Now, start the cv:  6
The accuracy in  this fold: 21.908602
Now, start the cv:  7
The accuracy in  this fold: 20.967742
Now, start the cv:  8
The accuracy in  this fold: 19.220430
Now, start the cv:  9
The accuracy in  this fold: 17.069892
Now, start the cv:  10
The accuracy in  this fold: 20.026882
2s Total Accuracy with RF (average):  19.986559139784944 1.3641645453997322
data_length = 1s
Now start 10 fold cross validation
Now, start the cv:  1
The accuracy in  this fold: 21.318090
Now, start the cv:  2
The accuracy in  this fold: 19.838601
Now, start the cv:  3


KeyboardInterrupt: ignored

In [ ]:
# RF
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
for length in times:
    cvscores = []
    print('data_length = ' + length)
    s_features_df, s_rf_features, s_svm_features, s_labels, s_features_column = load_data(length, 'stationary')
    h_features_df, h_rf_features, h_svm_features, h_labels, h_features_column = load_data(length, 'holding')
    print('Now start 10 fold cross validation')
    for i in range(1, 11):
        print('Now, start the cv: ',i)
        train_s_rf_features, test_s_rf_features, train_s_labels, test_s_labels = train_test_split(s_rf_features, s_labels, test_size = 0.1, shuffle = True)
        train_h_rf_features, test_h_rf_features, train_h_labels, test_h_labels = train_test_split(h_rf_features, h_labels, test_size = 0.1, shuffle = True)
        rf = RandomForestClassifier(n_estimators = 70, max_depth = 10, min_samples_leaf = 4, n_jobs = -1)
        rf.fit(train_h_rf_features, train_h_labels)
        pred = rf.predict(test_s_rf_features)
        scores = accuracy_score(test_s_labels, pred) * 100
        print('The accuracy in  this fold: %f'%scores)
        conf_mat = np.expand_dims(confusion_matrix(test_s_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
        cvscores.append(scores)
    print(length + " Total Accuracy with RF (average): ", np.mean(cvscores), np.std(cvscores))
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues, normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + '_train holding test stationary confusion_matrix.png')
plt.show()

# Inter Subject (same posture)

In [ ]:
# SVM
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
phase = ['stationary', 'holding']
#--------------------------------------------------------------------------------
for c in phase:
    conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
    for length in times:
        print('Now start 10 fold cross validation')
        print('phase = ' + c)
        print('data_length = ' + length)
        cvscores = []
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
        for folder in range(1, 11):
            print('Now, start the cv: ',folder)
            train_features_df, train_rf_features, train_svm_features, train_labels, train_features_column = load_random_subject(length, c, 'train', folder)
            test_features_df, test_rf_features, test_svm_features, test_labels, test_features_column = load_random_subject(length, c, 'test', folder)
            print('Number of training samples: ', len(train_svm_features))
            print('Number of testing samples: ', len(test_svm_features))
            svm = SVC(kernel = 'rbf', C = 8, gamma = 32, decision_function_shape = 'ovo')
            svm.fit(train_svm_features, train_labels)
            pred = svm.predict(test_svm_features)
            scores = accuracy_score(test_labels, pred)* 100
            print('The accuracy in this fold: %f'%scores)
            print("=============================")
            cvscores.append(scores)
            conf_mat = np.expand_dims(confusion_matrix(test_labels, pred), axis = 2)
            conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
        print(c + length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
        print("=============================================")
        print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
    conf_mat_sum = np.sum(conf_mat_all, axis = 2)
    plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                         normalize = True)
    plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + c + '_inter subject confusion_matrix.png')
    plt.show()

In [ ]:
# RF
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
phase = ['stationary', 'holding']
#--------------------------------------------------------------------------------
for c in phase:
    conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
    for length in times:
        print('Now start 10 fold cross validation')
        print('phase = ' + c)
        print('data_length = ' + length)
        cvscores = []
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
        for folder in range(1, 11):
            print('Now, start the cv: ',folder)
            train_features_df, train_rf_features, train_svm_features, train_labels, train_features_column = load_random_subject(length, c, 'train', folder)
            test_features_df, test_rf_features, test_svm_features, test_labels, test_features_column = load_random_subject(length, c, 'test', folder)
            rf = RandomForestClassifier(n_estimators = 70, max_depth = 10, min_samples_leaf = 4, n_jobs = -1)
            rf.fit(train_rf_features, train_labels)
            pred = rf.predict(test_rf_features)
            scores = accuracy_score(test_labels, pred)* 100
            print('The accuracy in this fold: %f'%scores)
            print("=============================")
            cvscores.append(scores)
            conf_mat = np.expand_dims(confusion_matrix(test_labels, pred), axis = 2)
            conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
        print(c + length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
        print("=============================================")
        print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
    conf_mat_sum = np.sum(conf_mat_all, axis = 2)
    plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                         normalize = True)
    # plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + c + '_inter subject confusion_matrix.png')
    plt.show()

Now start 10 fold cross validation
phase = stationary
data_length = 2s
Now, start the cv:  1
The accuracy in this fold: 48.452381
Now, start the cv:  2
The accuracy in this fold: 56.790123
Now, start the cv:  3
The accuracy in this fold: 59.507830
Now, start the cv:  4
The accuracy in this fold: 62.065095
Now, start the cv:  5
The accuracy in this fold: 62.111111
Now, start the cv:  6
The accuracy in this fold: 59.932660
Now, start the cv:  7
The accuracy in this fold: 49.333333
Now, start the cv:  8
The accuracy in this fold: 57.111111
Now, start the cv:  9
The accuracy in this fold: 60.000000
Now, start the cv:  10
The accuracy in this fold: 60.222222
stationary2s Total Accuracy (average):  57.5525867495666 4.635660811119011
Now start 10 fold cross validation
phase = stationary
data_length = 1s
Now, start the cv:  1
The accuracy in this fold: 46.607143
Now, start the cv:  2


KeyboardInterrupt: ignored

# Inter Subject (different posture)

In [ ]:
# SVM
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    print('Now start 10 fold cross validation')
    print('data_length = ' + length)
    cvscores = []
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
    for folder in range(1, 11):
        print('Now, start the cv: ',folder)
        train_features_df, train_rf_features, train_svm_features, train_labels, train_features_column = load_random_subject(length, 'holding', 'train', folder)
        test_features_df, test_rf_features, test_svm_features, test_labels, test_features_column = load_random_subject(length, 'stationary', 'test', folder)
        svm = SVC(kernel = 'rbf', C = 8, gamma = 32, decision_function_shape = 'ovo')
        svm.fit(train_svm_features, train_labels)
        pred = svm.predict(test_svm_features)
        scores = accuracy_score(test_labels, pred)* 100
        print('The accuracy in this fold: %f'%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(test_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#-------------------------------------------------------------------------------- 
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + 'train_holding_test_stationary ' + '_inter subject confusion_matrix.png')
plt.show()    

In [ ]:
# SVM
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    print('Now start 10 fold cross validation')
    print('data_length = ' + length)
    cvscores = []
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
    for folder in range(1, 11):
        print('Now, start the cv: ',folder)
        train_features_df, train_rf_features, train_svm_features, train_labels, train_features_column = load_random_subject(length, 'stationary', 'train', folder)
        test_features_df, test_rf_features, test_svm_features, test_labels, test_features_column = load_random_subject(length, 'holding', 'test', folder)
        svm = SVC(kernel = 'rbf', C = 8, gamma = 32, decision_function_shape = 'ovo')
        svm.fit(train_svm_features, train_labels)
        pred = svm.predict(test_svm_features)
        scores = accuracy_score(test_labels, pred)* 100
        print('The accuracy in this fold: %f'%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(test_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------     
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + 'train_stationary_test_holding ' + '_inter subject confusion_matrix.png')
plt.show()

In [ ]:
# RF
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    print('Now start 10 fold cross validation')
    print('data_length = ' + length)
    cvscores = []
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
    for folder in range(1, 11):
        print('Now, start the cv: ',folder)
        train_features_df, train_rf_features, train_svm_features, train_labels, train_features_column = load_random_subject(length, 'stationary', 'train', folder)
        test_features_df, test_rf_features, test_svm_features, test_labels, test_features_column = load_random_subject(length, 'holding', 'test', folder)
        rf = RandomForestClassifier(n_estimators = 70, max_depth = 10, min_samples_leaf = 4, n_jobs = -1)
        rf.fit(train_rf_features, train_labels)
        pred = rf.predict(test_rf_features)
        scores = accuracy_score(test_labels, pred)* 100
        print('The accuracy in this fold: %f'%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(test_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------    
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + 'train_stationary_test_holding' + '_inter subject confusion_matrix.png')
plt.show()

In [ ]:
# RF
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    print('Now start 10 fold cross validation')
    print('data_length = ' + length)
    cvscores = []
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
    for folder in range(1, 11):
        print('Now, start the cv: ',folder)
        train_features_df, train_rf_features, train_svm_features, train_labels, train_features_column = load_random_subject(length, 'holding', 'train', folder)
        test_features_df, test_rf_features, test_svm_features, test_labels, test_features_column = load_random_subject(length, 'stationary', 'test', folder)
        rf = RandomForestClassifier(n_estimators = 70, max_depth = 10, min_samples_leaf = 4, n_jobs = -1)
        rf.fit(train_rf_features, train_labels)
        pred = rf.predict(test_rf_features)
        scores = accuracy_score(test_labels, pred)* 100
        print('The accuracy in this fold: %f'%scores)
        print("=============================")
        cvscores.append(scores)
        conf_mat = np.expand_dims(confusion_matrix(test_labels, pred), axis = 2)
        conf_mat_all = np.concatenate([conf_mat_all, conf_mat], axis = 2)
    print(length + " Total Accuracy (average): ", np.mean(cvscores), np.std(cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------
conf_mat_sum = np.sum(conf_mat_all, axis = 2)
plot_confusion_matrix(conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + 'train_holding_test_stationary' + '_inter subject confusion_matrix.png')
plt.show()

# Combine (inter subject)

In [ ]:
# These list can be iterated throgh for loop
#--------------------------------------------------------------------------------
times = ['2s','1s','0.5s']
gestures = ['pat','stroke','grab','poke','scratch','notouch']
#--------------------------------------------------------------------------------
s_svm_conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
h_svm_conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
s_rf_conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
h_rf_conf_mat_all = np.expand_dims(np.zeros(shape = (6, 6)), axis = 2)
for length in times:
    print('Now start 10 fold cross validation')
    print('data_length = ' + length)
    s_svm_cvscores = []
    h_svm_cvscores = []
    s_rf_cvscores = []
    h_rf_cvscores = []
#--------------------------------------------------------------------------------
# this part is to load data and perform 10-fold cross validation
#--------------------------------------------------------------------------------
    for folder in range(1, 11):
        print('Now, start the cv: ',folder)
        s_train_features_df, s_train_rf_features, s_train_svm_features, s_train_labels, s_train_features_column = load_random_subject(length, 'stationary', 'train', folder)
        h_train_features_df, h_train_rf_features, h_train_svm_features, h_train_labels, h_train_features_column = load_random_subject(length, 'holding', 'train', folder)
        half_train_s_svm_features, half_train_s_svm_labels, half_train_h_svm_features, half_train_h_svm_labels = random_select(s_train_svm_features, s_train_labels, h_train_svm_features, h_train_labels)
        half_train_s_rf_features, half_train_s_rf_labels, half_train_h_rf_features, half_train_h_rf_labels = random_select(s_train_rf_features, s_train_labels, h_train_rf_features, h_train_labels)
        t_train_svm_features, t_train_svm_labels = concatenate(half_train_s_svm_features, half_train_h_svm_features,
                                                              half_train_s_svm_labels, half_train_h_svm_labels)
        t_train_rf_features, t_train_rf_labels = concatenate(half_train_s_rf_features, half_train_h_rf_features,
                                                      half_train_s_rf_labels, half_train_h_rf_labels)
        s_test_features_df, s_test_rf_features, s_test_svm_features, s_test_labels, s_train_features_column = load_random_subject(length, 'stationary', 'test', folder)
        h_test_features_df, h_test_rf_features, h_test_svm_features, h_test_labels, h_train_features_column = load_random_subject(length, 'holding', 'test', folder)
        svm = SVC(kernel = 'rbf', C = 8, gamma = 32, decision_function_shape = 'ovo')
        rf = RandomForestClassifier(n_estimators = 70, max_depth = 10, min_samples_leaf = 4, n_jobs = -1)
        svm.fit(t_train_svm_features, t_train_svm_labels)
        rf.fit(t_train_rf_features, t_train_rf_labels)
        s_svm_pred = svm.predict(s_test_svm_features)
        s_rf_pred = rf.predict(s_test_rf_features)
        h_svm_pred = svm.predict(h_test_svm_features)
        h_rf_pred = rf.predict(h_test_rf_features)
        s_svm_scores = accuracy_score(s_test_labels, s_svm_pred)* 100
        s_rf_scores = accuracy_score(s_test_labels, s_rf_pred)* 100
        h_svm_scores = accuracy_score(h_test_labels, h_svm_pred)* 100
        h_rf_scores = accuracy_score(h_test_labels, h_rf_pred)* 100
        print('The accuracy in this fold with SVM (test stationary): %f'%s_svm_scores)
        print('The accuracy in this fold with RF (test stationary): %f'%s_rf_scores)
        print("=============================")
        print('The accuracy in this fold with SVM (test holding): %f'%h_svm_scores)
        print('The accuracy in this fold with RF (test holding): %f'%h_rf_scores)
        s_svm_cvscores.append(s_svm_scores)
        s_rf_cvscores.append(s_rf_scores)
        h_svm_cvscores.append(h_svm_scores)
        h_rf_cvscores.append(h_rf_scores)
        # stationary confusion matrix
        s_svm_conf_mat = np.expand_dims(confusion_matrix(s_test_labels, s_svm_pred), axis = 2)
        s_svm_conf_mat_all = np.concatenate([s_svm_conf_mat_all, s_svm_conf_mat], axis = 2)
        s_rf_conf_mat = np.expand_dims(confusion_matrix(s_test_labels, s_rf_pred), axis = 2)
        s_rf_conf_mat_all = np.concatenate([s_rf_conf_mat_all, s_rf_conf_mat], axis = 2)
        # holding confusion matrix
        h_svm_conf_mat = np.expand_dims(confusion_matrix(h_test_labels, h_svm_pred), axis = 2)
        h_svm_conf_mat_all = np.concatenate([h_svm_conf_mat_all, h_svm_conf_mat], axis = 2)
        h_rf_conf_mat = np.expand_dims(confusion_matrix(h_test_labels, h_rf_pred), axis = 2)
        h_rf_conf_mat_all = np.concatenate([h_rf_conf_mat_all, h_rf_conf_mat], axis = 2)
    print('Test : stationary')
    print(length + " Total Accuracy with SVM (average): ", np.mean(s_svm_cvscores), np.std(s_svm_cvscores))
    print(length + " Total Accuracy with RF (average): ", np.mean(s_rf_cvscores), np.std(s_rf_cvscores))
    print('Test : holding')
    print(length + " Total Accuracy with SVM (average): ", np.mean(h_svm_cvscores), np.std(h_svm_cvscores))
    print(length + " Total Accuracy with RF (average): ", np.mean(h_rf_cvscores), np.std(h_rf_cvscores))
    print("=============================================")
    print("=============================================")
#--------------------------------------------------------------------------------
# this part is to plot confusion matrix
#--------------------------------------------------------------------------------
s_svm_conf_mat_sum = np.sum(s_svm_conf_mat_all, axis = 2)
s_rf_conf_mat_sum = np.sum(s_rf_conf_mat_all, axis = 2)
h_svm_conf_mat_sum = np.sum(h_svm_conf_mat_all, axis = 2)
h_rf_conf_mat_sum = np.sum(h_rf_conf_mat_all, axis = 2)
# plot confusion matrix (stationary)
plot_confusion_matrix(s_svm_conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + 'combine half test stationary_' + '_inter subject confusion_matrix.png')
plt.show()
plot_confusion_matrix(s_rf_conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + 'combine half test stationary_' + '_inter subject confusion_matrix.png')
plt.show()
# plot confusion matrix (holding)
plot_confusion_matrix(h_svm_conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'SVM_' + 'combine half test holding_' + '_inter subject confusion_matrix.png')
plt.show()
plot_confusion_matrix(h_rf_conf_mat_sum, classes = gestures, cmap = plt.cm.Blues,
                     normalize = True)
plt.savefig('/home/motionlab/Desktop/weitai/final_code/碩論final/SVM_RF/' + 'RF_' + 'combine half test holding_' + '_inter subject confusion_matrix.png')
plt.show()